# 👨‍💼👨‍🚀👨‍💻 Agents with Termination Strategies

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.18.0-alpha"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    //.AddOpenAIChatCompletion(
    //    modelId: Secrets.OpenAIModel,
    //    apiKey: Secrets.OpenAIApiKey)    
    .Build();

### 🧱 Our first agent gets the behavior of Jack Sparrow pirate.

In [ ]:
#pragma warning disable SKEXP0110

ChatCompletionAgent jackSparrowAgent = new() 
{ 
    Kernel = kernel,
    Instructions = """
        You are Jack Sparrow talking in Jack Sparrow style.
        Evaluate the context and reply to the last message by providing exactly one meaningful dialog line.
        The dialog line must be only one sentence of maximum 20 words.
        """,
    Description = "A chat bot that replies to the message in the voice of Jack Sparrow talking style.",
    Name = "JackSparrow"
};

### 🧱 The second agent impersonates Yoda.

In [ ]:
#pragma warning disable SKEXP0110

ChatCompletionAgent yodaAgent = new()
{
    Kernel = kernel,
    Instructions = """
        You are Yoda talking in Yoda style.
        Evaluate the context and reply to the last message by providing exactly one meaningful dialog line.
        The dialog line must be only one sentence of maximum 20 words.
        """,
    Description = "A chat bot that replies to the message in the voice of Yoda talking style.",
    Name = "Yoda"
};

### We want to create a poem contest.
### 🧱 Let's make an agent which impersonates a jury. The jury has to rate the poems written by Yoda and Jack Sparrow.

In [ ]:
#pragma warning disable SKEXP0110

ChatCompletionAgent juryAgent = new()
{
    Kernel = kernel,
    Instructions = """
        You are a jury able to rate poems.
        Evaluate each poem in the chat with a rating between 1 and 5, depending on how good the poem rhythms are.
        If there is a winner then always respond with 'WIN', then show the rating and name of each contestant, in descending order.
        if the ratings are equal then always respond with 'DEUCE', then show the rating and name of each contestant.
        Never add additional comments, stick to what I asked.
        """,
    Description = "A chat bot that evaluates poems and decide the winner.",
    Name = "Jury"
};

### Let's create a chat group for agents and add a goal to the chat.

In [ ]:
#pragma warning disable SKEXP0110

AgentGroupChat chat = new(jackSparrowAgent, yodaAgent, juryAgent)
{
    ExecutionSettings = new()
    {
        TerminationStrategy = new RegexTerminationStrategy(@"\b(WIN)\b")
        {
            Agents = [juryAgent],
            MaximumIterations = 50
        }
    }
};

var goal = """
    A poem contest starts between Yoda and Jack Sparrow. They have to write 4 lines poem.
    The contestants always respond with their poems, nothing else!
    """;

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, goal));


### See the chat arguments (the three agents!) along with the termination strategy and the selection strategy defined in execution settings.

### The termination strategy is a regex termination strategy checking for the word 'WIN'.

### ⚡Let the story (chat) begin!

In [ ]:
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0110

Console.WriteLine($">>>>>>> {AuthorRole.User} > {goal}.");

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"\n>>>>>>> {content.Role} [{content.AuthorName}] > {content.Content}");
}

Console.WriteLine($"IS COMPLETE: {chat.IsComplete}");

### Notice that the chat history is invoked for all attached agents.